In [ ]:
import torchxrayvision as xrv
import torchvision
import torch 
import skimage
import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [ ]:
pathologies = [  'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema',
                 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule',
                 'Mass', 'Hernia', 'Lung Lesion', 'Fracture','Lung Opacity','Enlarged Cardiomediastinum'
                ]

In [ ]:
data_transforms = torchvision.transforms.Compose([
        xrv.datasets.ToPILImage(),
        torchvision.transforms.RandomAffine(45, translate=(0.15, 0.15), scale=(0.85, 1.15)),
        torchvision.transforms.ToTensor(),xrv.datasets.XRayResizer(224)
    ])

In [ ]:
d_nih = xrv.datasets.NIH_Dataset(imgpath="C:\ML_C\CXR8\images",
                                 views=["PA","AP"], unique_patients=False, transform=data_transforms)

In [ ]:
d_chex = xrv.datasets.CheX_Dataset(imgpath="C:/ML_C/CheXpert/CheXpert-Keras-master/data/default_split/CheXpert-v1.0-small/CheXpert-v1.0-small",
                                   csvpath="C:/ML_C/CheXpert/CheXpert-Keras-master/data/default_split/CheXpert-v1.0-small/CheXpert-v1.0-small/train.csv",
                                 views=["PA","AP"], unique_patients=False, transform=data_transforms)

In [ ]:
print(d_nih.__class__)
xrv.datasets.relabel_dataset(pathologies, d_nih)
print(d_chex.__class__)
xrv.datasets.relabel_dataset(pathologies, d_chex)
dd = xrv.datasets.Merge_Dataset([d_nih, d_chex])

In [ ]:
# Use XRV transforms to crop and resize the images
transforms = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),
xrv.datasets.XRayResizer(224)])

# Load Google dataset and PyTorch dataloader
dataset =  xrv.datasets.NIH_Dataset(imgpath=r"C:/ML_C/CXR8/images",
                                 views=["PA","AP"], unique_patients=False, transform=data_transforms)   

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)
# Load pre-trained model and erase classifier
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.op_threshs = None # prevent pre-trained model calibration
model.classifier = torch.nn.Linear(1024,1) # reinitialize classifier

optimizer = torch.optim.Adam(model.classifier.parameters()) # only train classifier
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
# training loop
for batch in dataloader:
    outputs = model(batch["img"])
    targets = batch["lab"][:, dataset.pathologies.index("Atelectasis"), None]
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()